In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install tiktoken verovio accelerate -q

import os
import sys
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset


from datasets import Dataset as DS
from datasets import load_metric

import seaborn as sn

import matplotlib.pyplot as plt

import random

import time

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import gc

import inspect

try:
    from transformers import (
        AutoTokenizer,
        AutoModelForSeq2SeqLM,
        DataCollatorForSeq2Seq,
        Seq2SeqTrainer,
        Seq2SeqTrainingArguments,
        TrainingArguments,
        Trainer,
        pipeline,
        AutoModelForSequenceClassification
    )
except:
    !pip install transformers
    from transformers import (
        AutoTokenizer,
        AutoModelForSeq2SeqLM,
        DataCollatorForSeq2Seq,
        Seq2SeqTrainer,
        Seq2SeqTrainingArguments,
        pipeline
    )
    
from transformers import AutoConfig, AutoModel, AutoModelForSequenceClassification

print("All libraries have been installed successfully!", end="\r")

In [2]:
os.environ["WANDB_PROJECT"] = "bhashammo-distil-finetune"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints
os.environ["WANDB_API_KEY"] = "09cb24f7fe6eb5e62c4778d248d548aa64292375"
os.environ["WANDB_LOG_MODEL"] = "false"
os.environ["WANDB_WATCH"] = "all"

In [3]:
try:
    import wandb
    wandb.login()
except:
    !pip install wandb -q
    import wandb
    wandb.login()

wandb: Currently logged in as: smji14. Use `wandb login --relogin` to force relogin


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
data = pd.read_csv("/kaggle/input/muslim-hate-memes/final_ds.csv")

In [7]:
data

,index,image,content,label
0,0,images/0.jpg,The meme features a painting of Islamic schola...,1
1,1,images/1.jpg,The meme features a medieval-style illustratio...,1
2,2,images/10.jpg,The meme features a man with his hand raised i...,0
3,3,images/100.jpg,The meme features a cartoon image of a person ...,1
4,4,images/1000.jpg,The meme features a group of Muslim men in a m...,1
...,...,...,...,...
929,130,images/ugdv6px5hpn81.jpg,"The meme depicts a man with a distorted face, ...",1
930,131,images/75ekonjqh4t81.jpg,The meme depicts a cartoon of a man who is fas...,1
931,132,images/znpi8hx5nvd91.jpg,The meme depicts a scene from the animated tel...,0
932,133,images/6a6qkmyjt2j91.jpg,The meme depicts a person holding up a sign th...,0


In [8]:
data.drop(columns=['index'], inplace=True)

In [9]:
data.reset_index(drop=True, inplace=True)

In [10]:
data.columns

Index(['image', 'content', 'label'], dtype='object')

In [11]:
data

,image,content,label
0,images/0.jpg,The meme features a painting of Islamic schola...,1
1,images/1.jpg,The meme features a medieval-style illustratio...,1
2,images/10.jpg,The meme features a man with his hand raised i...,0
3,images/100.jpg,The meme features a cartoon image of a person ...,1
4,images/1000.jpg,The meme features a group of Muslim men in a m...,1
...,...,...,...
929,images/ugdv6px5hpn81.jpg,"The meme depicts a man with a distorted face, ...",1
930,images/75ekonjqh4t81.jpg,The meme depicts a cartoon of a man who is fas...,1
931,images/znpi8hx5nvd91.jpg,The meme depicts a scene from the animated tel...,0
932,images/6a6qkmyjt2j91.jpg,The meme depicts a person holding up a sign th...,0


In [12]:
def sanity_check():
    for i, row in data.iterrows():
        if not os.path.exists(f"/kaggle/input/muslim-hate-memes/{row['image']}"):
            print(row['image'],"does not exist")
        

In [13]:
sanity_check()

In [14]:
train_df, val_df = train_test_split(data, test_size=0.2, shuffle=True)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [15]:
val_df, test_df = train_test_split(val_df, test_size=0.5, shuffle=True)

In [16]:
test_df = test_df.reset_index(drop=True)

In [17]:
len(train_df), len(val_df), len(test_df)

(747, 93, 94)

In [18]:
torch.cuda.empty_cache()

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [20]:
from transformers import AutoModel, AutoTokenizer, BeitFeatureExtractor, BeitForMaskedImageModeling, AutoModelForPreTraining, BertTokenizerFast, VisualBertModel

In [21]:
from transformers import ViltProcessor, ViltModel

In [22]:
OCR_MODEL_NAME = "ucaslcl/GOT-OCR2_0"
MODEL_NAME = "dandelin/vilt-b32-mlm"

ocr_tokenizer = AutoTokenizer.from_pretrained(OCR_MODEL_NAME, trust_remote_code=True)
ocr_model = AutoModel.from_pretrained(OCR_MODEL_NAME, trust_remote_code=True, 
                                      low_cpu_mem_usage=True, device_map=device, use_safetensors=True, pad_token_id=ocr_tokenizer.eos_token_id)

ocr_model = ocr_model.eval().cuda()
ocr_model.generation_config.pad_token_id = ocr_tokenizer.pad_token_id

processor = ViltProcessor.from_pretrained(MODEL_NAME)
model = ViltModel.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/149 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [23]:
import torchvision.transforms as transforms

In [24]:
# res = ocr_model.chat(ocr_tokenizer, "/kaggle/input/muslim-hate-memes/images/0desqkyb24r51.jpeg", ocr_type='ocr')

In [25]:
# res

In [26]:
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding

In [27]:
# data_collator = DataCollatorWithPadding(tokenizer)

In [28]:
from datasets import Dataset as DS
from datasets import load_metric

In [29]:
embed_transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(           
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [30]:
from PIL import Image

In [31]:
# import torchvision

# embed_model = torchvision.models.resnext50_32x4d(weights='IMAGENET1K_V2')
# embed_model = embed_model.eval()
# embed_model = torch.nn.Sequential(*list(embed_model.children())[:-1])

In [32]:
# embed_model

In [33]:
class MuslimHateMemes(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        image_path = f"/kaggle/input/muslim-hate-memes/{row['image']}"
        label = int(row['label'])
        ocr_text = ocr_model.chat(ocr_tokenizer, image_path, ocr_type='ocr')
        image = Image.open(image_path).resize((252,252)).convert("RGB")
        inputs = processor(image, ocr_text, max_length=40, padding='max_length', truncation=True, return_tensors="pt")

        for x in inputs:
            inputs[x] = inputs[x].squeeze(0)
            # print(x,":",inputs[x].shape)
        
        return inputs, label, image_path

In [34]:
trainds = MuslimHateMemes(train_df)
valds = MuslimHateMemes(val_df)
testds = MuslimHateMemes(test_df)

In [35]:
trainloader = DataLoader(trainds, batch_size=16, shuffle=True)
valloader = DataLoader(valds, batch_size=2, shuffle=False)
testloader = DataLoader(testds, batch_size=2, shuffle=False)

In [36]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message=".*attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior.*")

In [37]:
# example = next(iter(trainloader))

In [38]:
# model

In [39]:
class MemeClassifer(nn.Module):
    def __init__(
        self,
        finetune: str = "limit",
        enc_finetune_limit: int = 2, 
        output: int = 1
    ):
        super().__init__()
        self.vilt = model
        self.clf = nn.Sequential(
            nn.LayerNorm(768),
            nn.Dropout(0.3),
            nn.Linear(768, 768, bias=True),
            nn.LayerNorm(768),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(768, output, bias=True),
        )
        self.act = nn.Sigmoid()

        if finetune == "all":
            for child in self.vilt.encoder.layer.children():
                for param in child.parameters():
                    param.requires_grad = True
        else:
            count = 0
            length = sum(1 for _ in self.vilt.encoder.layer.children())
            for child in self.vilt.encoder.layer.children():
                if count >= (length - enc_finetune_limit):
                    for param in child.parameters():
                        param.requires_grad = True
                else:
                    for param in child.parameters():
                        param.requires_grad = False
                count += 1

    def forward(self, inputs):
        
        outputs = self.vilt(**inputs)
        outputs = self.act(self.clf(outputs.pooler_output))

        return outputs

In [40]:
memeclsmodel = MemeClassifer(finetune="all", enc_finetune_limit=3)

In [41]:
memeclsmodel = memeclsmodel.to(device)

In [42]:
# memeclsmodel

In [43]:
def reset_weights(m):
    '''
        Try resetting model weights to avoid
        weight leakage.
      '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [44]:
!pip install evaluate -q

In [45]:
import evaluate

accuracy = evaluate.load('accuracy')
f1_metric = evaluate.load("f1", "binary")
precision_metric = evaluate.load("precision", "binary")
recall_metric = f1_macro = evaluate.load("recall", "binary")
# clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [46]:
def compute_metrics_v2(preds, labels):
    preds = preds.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    torch.cuda.empty_cache()
    
    acc = accuracy.compute(predictions=preds, references=labels)['accuracy']
    f1_macro = f1_metric.compute(predictions=preds, references=labels, average="macro")['f1']
    f1_micro = f1_metric.compute(predictions=preds, references=labels, average="micro")['f1']
    f1_weighted = f1_metric.compute(predictions=preds, references=labels, average="weighted")['f1']
    
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted', zero_division=1)['precision']
    
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted', zero_division=1)['recall']

    return {
        "accuracy": acc,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "f1_weighted": f1_weighted,
        "precision": precision,
        "recall": recall,
    }

In [47]:
optimizer = torch.optim.Adam(memeclsmodel.parameters(), lr=3e-4)
loss_function = torch.nn.BCELoss()

In [48]:
def training(model, training_loader):
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    example_ct = 0
    
    metrs = {
        "train/accuracy": 0,
        "train/f1_macro": 0,
        "train/f1_micro": 0,
        "train/f1_weighted": 0,
        "train/precision": 0,
        "train/recall": 0,
    }
    
    model.train()

    try:
        wandb.watch(model, log="all", log_freq=10)
    except:
        print("WANDB not logging")
    
    for _ , (data, labels, img_paths) in enumerate(training_loader):

        data = data.to(device)
        labels = labels.to(device, dtype = torch.float)

        outputs = model(data).squeeze(-1).float()
        loss = loss_function(outputs, labels)
        tr_loss += loss.item()
        big_idx = torch.round(outputs.data)
        metrics = compute_metrics_v2(big_idx, labels)
        
        for k,v in metrics.items():
            metrs[f"train/{k}"] += v
        
        example_ct += len(data)
        
        nb_tr_steps += 1
        nb_tr_examples+=labels.size(0)
        
        if _%1000==0:
            loss_step = tr_loss/len(training_loader)
#             wandb.log( mergeDicts([{"loss": loss_step}, metrics]) , step=nb_tr_steps)
            print(f"Training Loss per 1000 steps: {loss_step}")
            print(metrics)

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        
        torch.cuda.empty_cache()

    epoch_loss = tr_loss/len(training_loader)
    
    for k,v in metrs.items():
        metrs[k] = v / len(training_loader)

    print(f"Total Training: {epoch_loss}")
    print(f"Total Training Metrics:")
    
    print(metrs)

    return epoch_loss, metrs

In [49]:
def testing(model, testing_loader):
    model.eval()
    
    metrs = {
        "eval/accuracy": 0,
        "eval/f1_macro": 0,
        "eval/f1_micro": 0,
        "eval/f1_weighted": 0,
        "eval/precision": 0,
        "eval/recall": 0,
    }
    
    try:
        wandb.watch(model, log="all", log_freq=10)
    except:
        print("WANDB not logging")
    
    n_correct = 0; n_wrong = 0; total = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    with torch.no_grad():
        for _,(data, labels, img_paths) in enumerate(testing_loader):
            data = data.to(device)
            labels = labels.to(device, dtype = torch.float)
            
            outputs = model(data).squeeze(-1).float()
            loss = loss_function(outputs, labels)
            tr_loss += loss.item()
            big_idx = torch.round(outputs.data)
            metrics = compute_metrics_v2(big_idx, labels)
    
            for k,v in metrics.items():
                metrs[f"eval/{k}"] += v
            
            nb_tr_steps += 1
            nb_tr_examples+=labels.size(0)
            
            if _%1000==0:
                loss_step = tr_loss/len(testing_loader)
#                 wandb.log( mergeDicts([{"loss": loss_step}, metrs]) , step=nb_tr_steps)
                print(f"Validation Loss per 1000 steps: {loss_step}")
                print(metrics)
                
            torch.cuda.empty_cache()
                
    epoch_loss = tr_loss/len(testing_loader)
    for k,v in metrs.items():
        metrs[k] = v / len(testing_loader)
    
    print(f"Total Eval Loss: {epoch_loss}")
    print(f"Total Eval Metrics:")
    print(metrs)
    
    return epoch_loss, metrs

In [50]:
best_loss = 1e9

In [51]:
epochs = 10

In [52]:
with wandb.init(project="muslimhatememes", name='vilt run #4'):
    
    wandb.define_metric("epoch")
    wandb.define_metric("train/*", step_metric="epoch")
    wandb.define_metric("eval/*", step_metric="epoch")
    
    
    for epoch in range(epochs):
        print("==============================================================================")
        print("Starting epoch", epoch)
        
        loss, metrs = training(memeclsmodel, trainloader)
        eval_loss, eval_metrs = testing(memeclsmodel, valloader)
        
        print("Logging to W&B")
        wandb.log({"train/loss": loss} | {"eval/loss": eval_loss} | {"epoch": epoch} | metrs | eval_metrs, step=epoch)
        print("Done logging to W&B")
        
        if eval_loss < best_loss:
            best_loss = eval_loss
            torch.save(memeclsmodel.state_dict(), f'best_weights.pth')
        
        torch.save(memeclsmodel.state_dict(), f'checkpoint.pth')

    memeclsmodel.load_state_dict(torch.load("./best_weights.pth"))
    test_loss, test_metrs = testing(memeclsmodel, testloader)
    print("Test Loss:",test_loss)
    print("Test Metrics:")
    print(test_metrs)

wandb: wandb version 0.18.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241113_183653-kvp92kmv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run vilt run #4
wandb: ⭐️ View project at https://wandb.ai/smji14/muslimhatememes
wandb: 🚀 View run at https://wandb.ai/smji14/muslimhatememes/runs/kvp92kmv
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Starting epoch 0


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Training Loss per 1000 steps: 0.015594493835530382
{'accuracy': 0.625, 'f1_macro': 0.5, 'f1_micro': 0.625, 'f1_weighted': 0.5625, 'precision': 0.5892857142857143, 'recall': 0.625}
Total Training: 0.6513931028386379
Total Training Metrics:
{'train/accuracy': 0.6292311411992263, 'train/f1_macro': 0.5479275265884959, 'train/f1_micro': 0.6292311411992263, 'train/f1_weighted': 0.5898408263873282, 'train/precision': 0.6836330483936865, 'train/recall': 0.6292311411992263}
Validation Loss per 1000 steps: 0.01570719100059347
{'accuracy': 0.0, 'f1_macro': 0.0, 'f1_micro': 0.0, 'f1_weighted': 0.0, 'precision': 0.0, 'recall': 0.0}
Total Eval Loss: 0.600342350437286
Total Eval Metrics:
{'eval/accuracy': 0.6808510638297872, 'eval/f1_macro': 0.5957446808510637, 'eval/f1_micro': 0.6808510638297872, 'eval/f1_weighted': 0.6666666666666666, 'eval/precision': 0.9042553191489362, 'eval/recall': 0.6808510638297872}
Logging to W&B
Done logging to W&B
Starting epoch 1
Training Loss per 1000 steps: 0.013374881

wandb: ERROR Metric data exceeds maximum size of 10.4MB (11.0MB)
wandb: ERROR Summary data exceeds maximum size of 10.4MB. Dropping it.


...

wandb: ERROR Metric data exceeds maximum size of 10.4MB (14.4MB)


Training Loss per 1000 steps: 0.0008190392338215036
{'accuracy': 1.0, 'f1_macro': 1.0, 'f1_micro': 1.0, 'f1_weighted': 1.0, 'precision': 1.0, 'recall': 1.0}
Total Training: 0.07642337922601307
Total Training Metrics:
{'train/accuracy': 0.9735251450676983, 'train/f1_macro': 0.9725267146775154, 'train/f1_micro': 0.9735251450676983, 'train/f1_weighted': 0.9736490463789034, 'train/precision': 0.9761235071075496, 'train/recall': 0.9735251450676983}
Validation Loss per 1000 steps: 0.046561925969225294
{'accuracy': 0.5, 'f1_macro': 0.3333333333333333, 'f1_micro': 0.5, 'f1_weighted': 0.3333333333333333, 'precision': 0.75, 'recall': 0.5}
Total Eval Loss: 1.244817971700228
Total Eval Metrics:
{'eval/accuracy': 0.7021276595744681, 'eval/f1_macro': 0.6453900709219856, 'eval/f1_micro': 0.7021276595744681, 'eval/f1_weighted': 0.6950354609929078, 'eval/precision': 0.9308510638297872, 'eval/recall': 0.7021276595744681}
Logging to W&B
Done logging to W&B
Validation Loss per 1000 steps: 0.01291666893248

wandb: ERROR Metric data exceeds maximum size of 10.4MB (16.0MB)


Total Eval Loss: 0.620814219513472
Total Eval Metrics:
{'eval/accuracy': 0.6063829787234043, 'eval/f1_macro': 0.5106382978723403, 'eval/f1_micro': 0.6063829787234043, 'eval/f1_weighted': 0.5815602836879432, 'eval/precision': 0.9095744680851063, 'eval/recall': 0.6063829787234043}
Test Loss: 0.620814219513472
Test Metrics:
{'eval/accuracy': 0.6063829787234043, 'eval/f1_macro': 0.5106382978723403, 'eval/f1_micro': 0.6063829787234043, 'eval/f1_weighted': 0.5815602836879432, 'eval/precision': 0.9095744680851063, 'eval/recall': 0.6063829787234043}


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:             epoch ▁▂▃▃▄▅▆▆▇█
wandb:     eval/accuracy ▁▂▄▁▄▅▆█▅▂
wandb:     eval/f1_macro ▁▃▄▂▄▆▆█▅▄
wandb:     eval/f1_micro ▁▂▄▁▄▅▆█▅▂
wandb:  eval/f1_weighted ▂▁▃▂▃▆▇█▅▄
wandb:         eval/loss ▁▁▁▂▃▁▂▂▅█
wandb:    eval/precision ▃▃▅▁▅▆▆█▇▆
wandb:       eval/recall ▁▂▄▁▄▅▆█▅▂
wandb:    train/accuracy ▁▃▄▅▆▇████
wandb:    train/f1_macro ▁▃▅▅▆▇████
wandb:    train/f1_micro ▁▃▄▅▆▇████
wandb: train/f1_weighted ▁▃▅▅▆▇████
wandb:        train/loss █▇▆▆▄▃▂▁▁▁
wandb:   train/precision ▁▄▅▅▆▇████
wandb:      train/recall ▁▃▄▅▆▇████
wandb: 
wandb: Run summary:
wandb:             epoch 9
wandb:     eval/accuracy 0.70213
wandb:     eval/f1_macro 0.64539
wandb:     eval/f1_micro 0.70213
wandb:  eval/f1_weighted 0.69504
wandb:         eval/loss 1.24482
wandb:    eval/precision 0.93085
wandb:       eval/recall 0.70213
wandb:    train/accuracy 0.97353
wandb:    train/f1_macro 0

In [53]:
# k_folds = 10
# kfold = KFold(n_splits=k_folds, shuffle=True)

In [54]:
# dataset = ConcatDataset([trainds, valds])

In [55]:
# with wandb.init(project="muslimhatememes-kfold", name='vilt kfold cv k-10'):
    
#     wandb.define_metric("fold")
#     wandb.define_metric("train/*", step_metric="fold")
#     wandb.define_metric("test/*", step_metric="fold")
     
#     for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
#         print("==============================================================================")
    
#         print("Starting kfold",fold)
    
#         train_sampler = torch.utils.data.SubsetRandomSampler(train_ids)
#         test_sampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
#         train_loader = DataLoader(dataset, batch_size=16, sampler=train_sampler)
#         test_loader = DataLoader(dataset, batch_size=4, sampler=test_sampler)
    
#         loss, metrs = training(memeclsmodel, train_loader)
#         eval_loss, eval_metrs = testing(memeclsmodel, test_loader)
    
#         print("Logging to W&B")
#         wandb.log({"train/loss": loss} | {"test/loss": eval_loss} | {"fold": fold} | metrs | eval_metrs, step=fold)
#         print("Done logging to W&B")
    
#         if eval_loss < best_loss:
#             best_loss = eval_loss
#             torch.save(memeclsmodel.state_dict(), f'best_weights.pth')
    
#         torch.save(memeclsmodel.state_dict(), f'fold_{fold}_weights.pth')

In [56]:
# wandb.finish()

In [57]:
# testloader = DataLoader(testds, batch_size=2, shuffle=False)

In [58]:
# memeclsmodel.load_state_dict(torch.load("/kaggle/input/muslimhate-vilt/best_weights.pth"))

In [59]:
# test_loss, test_metrs = testing(memeclsmodel, testloader)

In [60]:
# print("Test Loss:",test_loss)

In [61]:
# print(test_metrs)

In [62]:
# for epoch in range(10):
#     print("==============================================================================")
#     print("Starting epoch", epoch)
    
#     loss, metrs = training(memeclsmodel, trainloader)
#     eval_loss, eval_metrs = testing(memeclsmodel, valloader)
    
#     print("Logging to W&B")
#     wandb.log({"train/loss": loss} | {"test/loss": eval_loss} | {"fold": fold} | metrs | eval_metrs, step=epoch)
#     print("Done logging to W&B")
    
#     if eval_loss < best_loss:
#         best_loss = eval_loss
#         torch.save(model.state_dict(), f'best_weights_{fold}.pth')
    
#     torch.save(model.state_dict(), f'fold_{fold}_weights.pth')

In [63]:
# import inspect

In [64]:
# print(inspect.getsource(model.attention.forward))

In [65]:
# print(inspect.getsource(model.forward))

In [66]:
# model

In [67]:
# print(inspect.getsource(model.get_extended_attention_mask))

In [68]:
# print(inspect.getsource(model.encoder.forward))

In [69]:
# with wandb.init(project="bhashammo-clf-train", name='full finetune distil-bert'):
#     for epoch in range(10):
#         print("Starting epoch",epoch)
        
#         loss, metrs = train(epoch)
#         wandb.log(mergeDicts([{"epoch": epoch + 1, "train/loss": loss}, metrs]), step=epoch)
        
#         eval_loss, eval_metrs = valid(model, val_loader)
#         wandb.log(mergeDicts([{"epoch": epoch + 1, "eval/loss": eval_loss}, eval_metrs]), step=epoch)

#         if eval_loss < best_loss:
#             best_loss = eval_loss
#             torch.save(model.state_dict(), 'best_weights.pth')

In [70]:
# model_id = "distil-bert-uncased-regional-bangla-text-to-reg-clf"

# training_args = TrainingArguments(
#     output_dir=model_id,
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=16,
#     eval_strategy="steps",
#     metric_for_best_model="f1_weighted",
#     load_best_model_at_end=True,
#     num_train_epochs=100,
#     save_steps=1000,
#     eval_steps=1000,
#     logging_steps=1000,
#     warmup_steps=1000,
#     learning_rate=1e-4,
#     weight_decay=1e-2,
#     save_total_limit=1,
#     push_to_hub=False,
#     lr_scheduler_type='reduce_lr_on_plateau',
#     fp16=True,
#     report_to="wandb",
#     run_name="distilbert-lr-on_plateau-warmup-1000"
# )

In [71]:
# torch.cuda.empty_cache()
# gc.collect()

In [72]:
# from transformers import EarlyStoppingCallback, IntervalStrategy

In [73]:
# trainer = Trainer(
#     model=model,
#     tokenizer=tokenizer,
#     args=training_args,
#     train_dataset=ds_train,
#     eval_dataset=ds_eval,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
# )


# trainer.train()


# torch.cuda.empty_cache()

# wandb.finish()

In [74]:
# trainer.save_model(model_id)

In [75]:
# trainer.push_to_hub("distil-bert-bengali-regional-classification")

In [76]:
# logs = trainer.state.log_history
# logs

In [77]:
# logsdf = pd.DataFrame(logs)
# logsdf.to_csv("logs.csv", index=False)

In [78]:
# torch.cuda.empty_cache()